### Import modules

In [ ]:
import pandas as pd

from diamondminer.miner import *

### Load Data

First you need to load your data; here is an example demo dataset.

> Note: Data is assumed to be clean and cropped so that it zooms into diamonds of interest.

In [ ]:
# Load the data from the txt file
demo_data_loaded = pd.read_csv('demo_data.txt', sep='\t', index_col=0)

# Convert the loaded data back to numpy arrays if needed
P_data = demo_data_loaded.columns.to_numpy().astype(float)
S_data = demo_data_loaded.index.to_numpy().astype(float)
I_data = demo_data_loaded.to_numpy().astype(float)

### Extract Diamonds

Next step is to extract the diamonds found in your data. The program does this by,

1. Breaking the data into two halfs: upper and lower.
2. Each half is used as input to the Hough transform.
3. Detected lines are then filtered for any duplicates and are sorted based on which section they belong
and whether their slope is positive or negative.
4. Finally, cluster the relevent lines together and "glue" them together to form our diamonds.

> Note: If the extraction isn't working, turn `debug=True` to plot any intermediate results. 
Adjust parameters found in `miner.filter_raw_data()`, `miner.extract_edges()`, `miner.extract_lines()` until lines are detected.

In [ ]:
miner = Miner(
    gate_data=P_data,
    ohmic_data=S_data * 1e-3,
    current_data=I_data
)
diamonds = miner.extract_diamonds(debug = False)

### Plotting the results!

In [ ]:
%matplotlib qt

fig, ax = plt.subplots()
ax.imshow(
    miner.current_data, 
    cmap='binary',
    aspect='auto',
    origin='lower',
    extent=[
        miner.gate_data[0],
        miner.gate_data[-1], 
        miner.ohmic_data[0], 
        miner.ohmic_data[-1]
    ],
)
ax.set_title("Coulomb Diamonds")
ax.set_xlabel("Gate Voltage (V)")
ax.set_ylabel("Ohmic Voltage (V)")
ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
for diamond in diamonds:
    diamond.print_summary()
    diamond.plot(ax)

plt.tight_layout()
plt.show()